<a href="https://colab.research.google.com/github/jbakery/Python-Portfolio/blob/main/Housing_Valuation_Grid_search_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Upload the data file to Google Colab.

from google.colab import files

uploaded = files.upload()

Saving Melbourne_housing_FULL.csv to Melbourne_housing_FULL.csv


In [1]:
# Import libaries, including GridSearchCV

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV



In [10]:
# Insert Data into Data Frame and print first 5 rows.

df = pd.read_csv('Melbourne_housing_FULL.csv')

df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [11]:
# Delete uneeded columns

del df['Address']
del df['Method']
del df['SellerG']
del df['Date']
del df['Postcode']
del df['Lattitude']
del df['Longtitude']
del df['Regionname']
del df['Propertycount']


In [12]:
# Remove rows with missing values

df.dropna( axis = 0, how = 'any', subset = None, inplace= True)

df.head()

,Suburb,Rooms,Type,Price,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea
2,Abbotsford,2,h,1035000.0,2.5,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra City Council
4,Abbotsford,3,h,1465000.0,2.5,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra City Council
6,Abbotsford,4,h,1600000.0,2.5,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra City Council
11,Abbotsford,3,h,1876000.0,2.5,4.0,2.0,0.0,245.0,210.0,1910.0,Yarra City Council
14,Abbotsford,2,h,1636000.0,2.5,2.0,1.0,2.0,256.0,107.0,1890.0,Yarra City Council


In [13]:
# Convert non-numeric data using one-hot encoding

df = pd.get_dummies(df, columns = ['Suburb','CouncilArea','Type'])

df.head()

,Rooms,Price,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Suburb_Abbotsford,...,CouncilArea_Port Phillip City Council,CouncilArea_Stonnington City Council,CouncilArea_Whitehorse City Council,CouncilArea_Whittlesea City Council,CouncilArea_Wyndham City Council,CouncilArea_Yarra City Council,CouncilArea_Yarra Ranges Shire Council,Type_h,Type_t,Type_u
2,2,1035000.0,2.5,2.0,1.0,0.0,156.0,79.0,1900.0,1,...,0,0,0,0,0,1,0,1,0,0
4,3,1465000.0,2.5,3.0,2.0,0.0,134.0,150.0,1900.0,1,...,0,0,0,0,0,1,0,1,0,0
6,4,1600000.0,2.5,3.0,1.0,2.0,120.0,142.0,2014.0,1,...,0,0,0,0,0,1,0,1,0,0
11,3,1876000.0,2.5,4.0,2.0,0.0,245.0,210.0,1910.0,1,...,0,0,0,0,0,1,0,1,0,0
14,2,1636000.0,2.5,2.0,1.0,2.0,256.0,107.0,1890.0,1,...,0,0,0,0,0,1,0,1,0,0


In [15]:
# Assign X and y Variables

X = df.drop('Price',axis=1)
y = df['Price']

In [16]:
# Split dat into test/train set (70/30 split) and shuffle

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, shuffle = True)

In [17]:
# Input algorithm

model = ensemble.GradientBoostingRegressor()


In [23]:
# Set the configureations that you wish to test. To minimize processing time, limit num. of variable or experiment on each hyperparameter separately.

hyperparameters = {
    'n_estimators' : [200,300],
    'max_depth' : [4, 6],
    'min_samples_leaf' : [5, 6],
    'learning_rate' : [0.01, 0.02],
    'max_features' : [0.8, 0.9],
    'loss' : ['ls', 'lad', 'huber']
}

In [24]:
# Define grid search. Run with four CPUs in parallel if applicable.

grid = GridSearchCV(model, hyperparameters, n_jobs= 4)


In [26]:
# Run grid search on training data

grid.fit(X_train, y_train)




/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
320 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
102 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py", line 420, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/

GridSearchCV(estimator=GradientBoostingRegressor(), n_jobs=4,
             param_grid={'learning_rate': [0.01, 0.02],
                         'loss': ['ls', 'lad', 'huber'], 'max_depth': [4, 6],
                         'max_features': [0.8, 0.9], 'min_samples_leaf': [5, 6],
                         'n_estimators': [200, 300]})

In [27]:
# Return optimal hyperparameters

grid.best_params_

{'learning_rate': 0.02,
 'loss': 'huber',
 'max_depth': 6,
 'max_features': 0.8,
 'min_samples_leaf': 5,
 'n_estimators': 300}

In [28]:
# Check model for accuracy using optimal hyperparameters

mae_train = mean_absolute_error(y_train, grid.predict(X_train))

print("Test Set Mean Absolute Error: %.2f" % mae_train)

mae_test = mean_absolute_error(y_test, grid.predict(X_test))

print("Test Set Mean Absolute Error: %.2f" % mae_test)

Test Set Mean Absolute Error: 139641.80
Test Set Mean Absolute Error: 161319.08
